In [ ]:
import math

def round_to_nearest(num):
        return math.floor(num / 5) * 5

print(round_to_nearest(-1))

In [ ]:
#aggregation from individual csv files

import pandas as pd
import math
from os import listdir
from os.path import isfile, join

def round_to_nearest(num):
        return round(num * 4) / 4

allgrouped = None
files = [f for f in listdir("F:/Datasets/old uncertain CSVs/") if isfile(join("F:/Datasets/old uncertain CSVs/", f))]

for x in range(len(files)):
        df = pd.read_csv('CSVs/'+str(files[x]))
        df = df.drop(df.columns[[0]],axis = 1)
        df = df.drop(columns=["Time"])
        df['Month'] = df['Date'].str[:5]+'-'+df['Date'].str[5:7]
        df['Month'] = df['Month'].str.replace('/','')
        df = df.drop(columns=["Date"])

        df['Latitude'] = round_to_nearest(df['Latitude'])
        df['Longitude'] = round_to_nearest(df['Longitude'])
        df = df[(df['Power (J)'] > 1)] 

        grouped_df = df.groupby(['Latitude', 'Longitude','Month']).agg({'Power (J)': ['mean', 'min', 'max','count']})
        grouped_df.columns = ['power_mean', 'power_min', 'power_max','lightning_count']
        grouped_df = grouped_df.reset_index()
        grouped_df = grouped_df.sort_values(by=['Month'])

        allgrouped = pd.concat([allgrouped,grouped_df],ignore_index=True)

allgrouped['Month'] = allgrouped['Month'].str.replace('12','d')
allgrouped['Month'] = allgrouped['Month'].str.replace('-1','-01')
allgrouped['Month'] = allgrouped['Month'].str.replace('-2','-02')
allgrouped['Month'] = allgrouped['Month'].str.replace('d','12')
allgrouped = allgrouped.sort_values(by=['Month'])

allgrouped.to_csv("grouped025no0j.csv")

In [2]:
#all to one csv no aggregation - mediterranean
from global_land_mask import globe
import pandas as pd
import math
from os import listdir
from os.path import isfile, join

allgrouped = None
files = [f for f in listdir("F:/Datasets/old uncertain CSVs/") if isfile(join("F:/Datasets/old uncertain CSVs/", f))]

for x in range(len(files)):
    df = pd.read_csv('../old uncertain CSVs/'+str(files[x]))
    df = df.drop(df.columns[[0,2,5,6,8,9]],axis = 1)

    #df = df.drop(df[df.Longitude > 180].index)
    df = df.drop(df[globe.is_land(df.Latitude, df.Longitude)].index)

    middleEast = df.copy()
    middleEast = middleEast[(middleEast['Latitude'] < 50) & (middleEast['Latitude'] > 29)] 
    middleEast = middleEast[(middleEast['Longitude'] > -5) & (middleEast['Longitude'] < 36)]
    middleEast = middleEast.drop(middleEast[((middleEast['Latitude'] > 40.625) & (middleEast['Longitude'] > 26.625))].index)
    middleEast = middleEast.drop(middleEast[((middleEast['Latitude'] > 43) & (middleEast['Longitude'] < 2))].index)

    allgrouped = pd.concat([allgrouped,middleEast],ignore_index=True)

allgrouped.to_csv("../postprocessed/ww no land lightnings 2009-2021.csv")

In [ ]:
#all to one csv no aggregation - global
from global_land_mask import globe
import pandas as pd
import math
from os import listdir
from os.path import isfile, join

allgrouped = None
files = [f for f in listdir("F:/Datasets/preprocessed/") if isfile(join("F:/Datasets/preprocessed/", f))]

for x in range(len(files)):
    df = pd.read_csv('../preprocessed/'+str(files[x]))
    df = df.drop(df.columns[[0,2,5,6,8,9]],axis = 1)

    df = df.drop(df[df.Longitude > 180].index)
    df = df.drop(df[globe.is_land(df.Latitude, df.Longitude)].index)

    #df = df[df["Power (J)"]>1000000]

    allgrouped = pd.concat([allgrouped,df],ignore_index=True)

allgrouped.to_csv("ww no land lightnings.csv")

In [5]:
allgrouped

,Date,Latitude,Longitude,Power (J)
0,2009/12/2,-12.6641,30.7067,1005865.74
1,2009/12/2,9.2379,70.2729,1323520.19
2,2009/12/3,-16.2751,-69.3359,1126573.76
3,2009/12/5,43.8357,136.0687,1507004.49
4,2009/12/5,6.6969,58.4743,1050767.01
...,...,...,...,...
10206,2022/1/22,-38.0945,-124.8827,9626721.71
10207,2022/1/22,-19.8285,128.8518,1992202.40
10208,2022/1/23,-35.2051,-51.7961,2328116.37
10209,2022/1/23,-33.1053,163.0162,1021775.12
